 # Opis notatnika
 Na bazie danych, gdzie umieszczone są już dane, wszystko zawarte jest na schemacie `public`. Ponieważ zgodnie z wymaganiami otrzymujemy nowy obszar wykorzystania danych, stworzymy sobie schemat dedykowany - `reporting`.  
 Dalej stworzymy widoki, które odpowiedzą na zadane wcześniej pytania.


 Zaimportowanie potrzebnych bibliotek

In [1]:
import psycopg2

 ## Konfiguracja połączenia


 Zdefiniowanie zmiennych `con` oraz `cursor` do wprowadzania poleceń do bazy danych

In [2]:
con = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='al2cx7hsixj',  # Przypadkowe, nieprawdziwe hasło
    dbname='airlines', 
)
cursor = con.cursor()

 ## Wczytanie pliku `reporting.sql`


In [3]:
# Wczytanie pliku jako string. Plik zawiera już napisane komendy tworzące tabele, gotowe do uruchomienia:

with open(r"C:\Coderslab_Analityk_kurs\Projekt_koncowy\sql\reporting.sql", "r", encoding="utf-8") as sql_file:
    sql_content = sql_file.read()
    sql_content = sql_content.replace('\n', ' ')

In [ ]:
'''
Treść pliku reporting.sql:

    FROM reporting.top_reliability_roads
    join public.airport_list AS al ON reporting.top_reliability_roads.origin_airport_id = al.origin_airport_id
    join public.airport_list AS al2 ON reporting.top_reliability_roads.dest_airport_id = al2.origin_airport_id
    WHERE cnt >= 10000;

    /*
    Tworzenie widoku z rankowaniem
    */
    
    CREATE OR REPLACE VIEW reporting.top_reliability_roads3 AS
    SELECT 
    origin_airport_id,
    origin_airport_name,
    dest_airport_id,
    dest_airport_name,
    cnt,
    year,
    reliability,
    DENSE_RANK() OVER ( ORDER BY reliability) AS nb
    FROM reporting.top_reliability_roads2
    ;
    
    
    /*
    Definicja widoku reporting.year_to_year_comparision, który będzie zawierał następujące kolumny:
    - `year`
    - `month`,
    - `flights_amount`
    - `reliability`
    */
    
    CREATE VIEW reporting.year_to_year_comparision AS
    SELECT 
    year,
    month, 
    COUNT(id) AS flights_amount,
    ROUND(100*CAST (SUM(is_delayed) AS numeric) / CAST (COUNT(id) AS numeric)) AS reliability
    FROM reporting.flight
    GROUP BY year, month
    ;
    
    
    /*
    Definicja widoku reporting.day_to_day_comparision, który będzie zawierał następujące kolumny:
    - `year`
    - `day_of_week`
    - `flights_amount`
    */
    
    CREATE OR REPLACE VIEW reporting.day_to_day_comparision AS
    SELECT 
    year,
    day_of_week, 
    COUNT(id) AS flights_amount
    FROM reporting.flight
    GROUP BY year, day_of_week
    ;


    /*
    Definicja widoku reporting.day_by_day_reliability, ktory będzie zawierał następujące kolumny:
    - `date` jako złożenie kolumn `year`, `month`, `day`, powinna być typu `date`
    - `reliability` jako odsetek opóźnień danego dnia

    
    */
    CREATE OR REPLACE VIEW reporting.day_by_day_reliability AS
    SELECT
    TO_DATE(year || '-' || month || '-' || day_of_month, 'YYYY-MM-DD') AS date,
    ROUND(100*CAST (SUM(is_delayed) AS numeric) / CAST (COUNT(id) AS numeric)) AS reliability
    FROM reporting.flight
    GROUP BY date

'''

 Rozdzielenie zawartość pliku `reporting.sql` na mniejsze kwerendy używając `;`

In [4]:
kwer = sql_content.split(";")


# Sprawdzenie liczby znalezionych poleceń:

len(kwer)

11

 Wykonanie każdej z kwerend, aby zainicjować strukturę bazy danych

In [5]:
for k in range(len(kwer)): 
    print("Executing query nr: ", k)  # W celu monitorowania które polecenie jest wykonywane
    cursor.execute(kwer[k])

Executing query nr:  0
Executing query nr:  1
Executing query nr:  2
Executing query nr:  3
Executing query nr:  4
Executing query nr:  5
Executing query nr:  6
Executing query nr:  7
Executing query nr:  8
Executing query nr:  9
Executing query nr:  10


 Zatwierdzenie wszystkich operacji na bazie, czyli stworzenie widoków

In [6]:
con.commit()

 # Podsumownie
 W tym notatniku stworzono nowy schemat - `reporting`. Wykorzystanie go może stanowić dobrą bazę do wizualizacji za pomocą dashboardów.